Please enter your provided API key and password in the following cell. 

In [132]:
API_KEY = 'XZNYTFQAITSJYCDP'
username = 'team49'

In [139]:
URL='http://ec2-13-59-143-196.us-east-2.compute.amazonaws.com:8080'


In [134]:
from collections import OrderedDict

class OrderBook:
    def __init__(self, raw_order_book: dict = None):
        if raw_order_book is None:
            raw_order_book = {}
        if not isinstance(raw_order_book, dict):
            raise TypeError("Input data must be a dictionary.")

        self.order_books = {}
        for ticker, volumes in raw_order_book.items():
            self.order_books[ticker] = {
                'bidVolumes': self._create_sorted_dict(volumes.get('bidVolumes', {}), reverse=True),
                'askVolumes': self._create_sorted_dict(volumes.get('askVolumes', {}), reverse=False)
            }

    def _create_sorted_dict(self, volumes: dict, reverse: bool):
        return OrderedDict(sorted(
            ((float(price), float(qty)) for price, qty in volumes.items()),
            key=lambda x: x[0],
            reverse=reverse
        ))

    def update_volumes(self, updates: list):
        if not isinstance(updates, list):
            raise TypeError("Updates must be provided as a list.")

        for update in updates:
            if not isinstance(update, dict) or \
               'ticker' not in update or 'price' not in update or 'side' not in update or 'volume' not in update:
                raise ValueError("Each update must be a dictionary with keys 'ticker', 'price', 'side', and 'volume'.")

            ticker = update['ticker']
            price = float(update['price'])
            side = update['side'].upper()
            volume = float(update['volume'])

            if ticker not in self.order_books:
                self.order_books[ticker] = {
                    'bidVolumes': OrderedDict(),
                    'askVolumes': OrderedDict()
                }

            if side == 'BID':
                if volume == 0.0:
                    self.order_books[ticker]['bidVolumes'].pop(price, None)
                else:
                    self.order_books[ticker]['bidVolumes'][price] = volume
                    self.order_books[ticker]['bidVolumes'] = self._create_sorted_dict(self.order_books[ticker]['bidVolumes'], reverse=True)
            elif side == 'ASK':
                if volume == 0.0:
                    self.order_books[ticker]['askVolumes'].pop(price, None)
                else:
                    self.order_books[ticker]['askVolumes'][price] = volume
                    self.order_books[ticker]['askVolumes'] = self._create_sorted_dict(self.order_books[ticker]['askVolumes'], reverse=False)
            else:
                raise ValueError("Side must be 'BID' or 'ASK'.")

    def __repr__(self):
        return f"OrderBook({self.order_books})"
    def __str__(self):
        output = []
        for ticker, data in self.order_books.items():
            output.append(f"Ticker: {ticker}")
            output.append("  Bid Volumes:")
            for price, volume in data['bidVolumes'].items():
                output.append(f"    {price:.2f}: {volume:.2f}")
            output.append("  Ask Volumes:")
            for price, volume in data['askVolumes'].items():
                output.append(f"    {price:.2f}: {volume:.2f}")
        return "\n".join(output)


In [135]:
import json
import websocket
import threading
from websocket import WebSocketApp
import time
import os
from IPython.display import clear_output

class WebSocketClient:
    def __init__(self, order_book: OrderBook):
        self.ws_url = "ws://ec2-13-59-143-196.us-east-2.compute.amazonaws.com:8080/exchange-socket"
        self.connected = False
        self.ws = None
        self.order_book = order_book
    def on_message(self, ws, message):
        """Handle incoming WebSocket messages."""
        try:
            if isinstance(message, bytes):
                message = message.decode('utf-8')

            if '\n\n' in message:
                headers, body = message.split('\n\n', 1)
                body = body.replace('\x00', '').strip()
                json_body = json.loads(body)

                if "content" in json_body:
                    content = json.loads(json_body["content"])
                    if isinstance(content, list):
                        self.order_book.update_volumes(content)
                    clear_output(wait=True)
                    print(self.order_book)
        except Exception as e:
            pass

    def on_error(self, ws, error):
        print(f"Error: {error}")

    def on_open(self, ws):
        print("WebSocket connection established")
        # Send STOMP CONNECT frame
        connect_frame = "CONNECT\naccept-version:1.1,1.0\nhost:localhost\n\n\x00"
        ws.send(connect_frame)

        # Subscribe to orderbook topic
        subscribe_frame = "SUBSCRIBE\nid:sub-0\ndestination:/topic/orderbook\nack:auto\n\n\x00"
        ws.send(subscribe_frame)

        self.connected = True
        print("STOMP connection and subscription established")

    def on_close(self, ws, close_status_code, close_msg):
        print(f"Disconnected: {close_msg if close_msg else 'No message'}")
        self.connected = False

    def connect(self):
        """Connect to the WebSocket STOMP broker"""
        websocket.enableTrace(False)
        self.ws = WebSocketApp(
            self.ws_url,
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close
        )

        # Start WebSocket connection in a separate thread
        wst = threading.Thread(target=self.ws.run_forever)
        wst.daemon = True
        wst.start()

    def disconnect(self):
        """Disconnect from the STOMP broker"""
        if self.connected and self.ws:
            # Send STOMP DISCONNECT frame
            disconnect_frame = "DISCONNECT\nreceipt:77\n\n\x00"
            self.ws.send(disconnect_frame)
            self.ws.close()
            print("Disconnected from broker")





In [136]:
import urllib
import threading
import json
import time

class TradingClient:
    def __init__(self, username, api_key):
        self.username = username
        self.api_key = api_key
        
        # Initialize stop_flag and socket_thread
        self.stop_flag = False
        self.socket_thread = None
        
        self.user_buildup()
        self.run_socket()

    def user_buildup(self):
        """Authenticate the user and obtain a session token."""
        form_data = {
            'username': self.username,
            'apiKey': self.api_key
        }
        req = urllib.request.Request(
            URL + '/buildup',
            data=json.dumps(form_data).encode('utf-8'),
            method='POST'
        )
        req.add_header('Content-Type', 'application/json')
        response = json.loads(urllib.request.urlopen(req).read().decode('utf-8'))
        self.session_token = response.get('sessionToken')
        self.order_book = OrderBook(json.loads(response['orderBookData']))
        return response

    def place_limit(self, ticker, volume, price, is_bid):
        """Place a Limit Order on the exchange."""
        if not self.session_token:
            raise Exception("User not authenticated. Call user_buildup first.")
import urllib
import threading
import json
import time

class TradingClient:
    def __init__(self, username, api_key):
        self.username = username
        self.api_key = api_key
        
        # Initialize stop_flag and socket_thread
        self.stop_flag = False
        self.socket_thread = None
        
        self.user_buildup()
        self.run_socket()
        self.run_algorithm_thread()
    def user_buildup(self):
        """Authenticate the user and obtain a session token."""
        form_data = {
            'username': self.username,
            'apiKey': self.api_key
        }
        req = urllib.request.Request(
            URL + '/buildup',
            data=json.dumps(form_data).encode('utf-8'),
            method='POST'
        )
        req.add_header('Content-Type', 'application/json')
        response = json.loads(urllib.request.urlopen(req).read().decode('utf-8'))
        self.session_token = response.get('sessionToken')
        self.order_book = OrderBook(json.loads(response['orderBookData']))
        return response

    def place_limit(self, ticker, volume, price, is_bid):
        """Place a Limit Order on the exchange."""
        if not self.session_token:
            raise Exception("User not authenticated. Call user_buildup first.")

        form_data = {
            'username': self.username,
            'sessionToken': self.session_token,
            'ticker': ticker,
            'volume': volume,
            'price': price,
            'isBid': is_bid
        }
        req = urllib.request.Request(
            URL + '/limit_order',
            data=json.dumps(form_data).encode('utf-8'),
            method='POST'
        )
        req.add_header('Content-Type', 'application/json')
        return json.loads(urllib.request.urlopen(req).read().decode('utf-8'))

    def place_market(self, ticker, volume, is_bid):
        """Place a Market Order on the exchange."""
        if not self.session_token:
            raise Exception("User not authenticated. Call user_buildup first.")

        form_data = {
            'username': self.username,
            'sessionToken': self.session_token,
            'ticker': ticker,
            'volume': volume,
            'isBid': is_bid
        }
        req = urllib.request.Request(
            URL + '/market_order',
            data=json.dumps(form_data).encode('utf-8'),
            method='POST'
        )
        req.add_header('Content-Type', 'application/json')
        content = json.loads(urllib.request.urlopen(req).read().decode('utf-8'))
        print(content)
    def run_socket(self):
        if not hasattr(self.order_book, '__class__') or self.order_book.__class__.__name__ != "OrderBook":
            print(self.order_book)
            raise Exception("Bad input - must be an instance of the OrderBook class")
    
        self.client = WebSocketClient(self.order_book)

        def socket_runner():
            self.client.connect()
            while not self.stop_flag:
                time.sleep(2.0)
            # Optionally, you can do cleanup here after the loop ends
            # e.g., self.client.disconnect()

        # Store the thread in self.socket_thread
        self.socket_thread = threading.Thread(target=socket_runner, daemon=True)
        self.socket_thread.start()

    def stop(self):
        """Stop the WebSocket thread and clean up."""
        print("Stopping the WebSocket connection...")
        if not self.socket_thread:
            print("WebSocket thread has not been started.")
            return
        
        # First disconnect from the broker
        self.client.disconnect()
        
        # Signal the stop event
        self.stop_flag = True

        # If the thread is still active, join it
        if self.socket_thread.is_alive():
            self.socket_thread.join()
        print("WebSocket connection stopped.")
        def teardown(username, session_token):
            form_data = {
                'username': username,
                'sessionToken': session_token
            }
            req = urllib.request.Request(URL + '/teardown', data=json.dumps(form_data).encode('utf-8'), method='POST')
            req.add_header('Content-Type', 'application/json')
            return json.loads(urllib.request.urlopen(req).read().decode('utf-8'))
        teardown(self.username, self.session_token)
    def algorithm(self):
        while not self.stop_flag:
            for ticker, data in self.order_book.order_books.items():
                if data['askVolumes'] and data['bidVolumes']:
                    ask = next(iter(data['askVolumes']))
                    bid = next(iter(data['bidVolumes']))
                    if ask - bid < 100:
                        self.place_market(ticker, 10, True)
            time.sleep(0.5)
        
    def run_algorithm_thread(self):
        def loop_algorithm():
            self.algorithm()  # same logic as before
        self.algorithm_thread = threading.Thread(target=loop_algorithm, daemon=True)
        self.algorithm_thread.start()




In [138]:
trading_client = TradingClient(username, API_KEY)

HTTPError: HTTP Error 405: Method Not Allowed

In [131]:
trading_client.stop()

Stopping the WebSocket connection...
Disconnected from broker
WebSocket connection stopped.
